In [ ]:
# !pip install pvlib rasterio geopandas shapely pyproj opencv-python scikit-image scipy pandas -q
print("✅ Installed (if needed)")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import rasterio
from rasterio import features

import geopandas as gpd
from shapely.geometry import shape, Point
from shapely.ops import unary_union
from pyproj import Transformer

import pvlib

# ----------------------------
# INPUTS (edit these)
# ----------------------------
IMAGE_TIF = r"C:\...\image.tif"      # GeoTIFF RGB (or multi-band)
MASK_TIF  = r"C:\...\mask.tif"       # GeoTIFF binary mask (1=building, 0=background), same grid as IMAGE
OUTPUT_DIR = r"C:\...\pv_outputs"
# ----------------------------

# PV assumptions (editable)
ROOF_USABLE_FRACTION = 0.70     # part of roof usable (setbacks, obstacles)
MODULE_EFF = 0.20               # 20% typical module efficiency (assumption)
POWER_DENSITY_W_PER_M2 = 200    # ≈ 200 W/m² consistent with ~20% at 1000 W/m² (assumption)
SYSTEM_LOSS_FRACTION = 0.14     # 14% overall losses (soiling+wiring+mismatch etc., assumption)

# PVWatts params
GAMMA_PDC = -0.0035             # 1/C typical range -0.002..-0.005 :contentReference[oaicite:2]{index=2}
INV_EFF_NOM = 0.96              # nominal inverter eff :contentReference[oaicite:3]{index=3}

# Location / weather simplification
TZ = "UTC"                      # if you know local tz, set it
ALTITUDE_M = 0                  # if known, set it

# Orientation sweep
TILT_GRID = list(range(0, 46, 5))         # 0..45 step 5
AZI_GRID  = list(range(0, 360, 10))       # 0..350 step 10

# Shading model (simple, parametric)
USE_SHADING = True
DEFAULT_BUILDING_HEIGHT_M = 10.0
HEIGHT_BY_AREA = True   # crude: taller if footprint bigger (optional)
HEIGHT_MIN_M = 6
HEIGHT_MAX_M = 25


In [ ]:
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

with rasterio.open(IMAGE_TIF) as src:
    img = src.read()             # (bands, H, W)
    img_profile = src.profile
    crs = src.crs
    transform = src.transform
    bounds = src.bounds

with rasterio.open(MASK_TIF) as msrc:
    mask = msrc.read(1).astype(np.uint8)
    mask_profile = msrc.profile

H, W = mask.shape
print("Image bands:", img.shape, " Mask:", mask.shape)
print("CRS:", crs)
print("Bounds:", bounds)

# Display (use first 3 bands if possible)
img_rgb = np.transpose(img[:3], (1,2,0)) if img.shape[0] >= 3 else np.repeat(img[0][...,None], 3, axis=2)
# Normalize for display only
img_disp = img_rgb.astype(np.float32)
img_disp -= img_disp.min()
img_disp /= (img_disp.max() + 1e-8)

plt.figure(figsize=(14,5))
plt.subplot(1,3,1); plt.imshow(img_disp); plt.title("Image"); plt.axis("off")
plt.subplot(1,3,2); plt.imshow(mask, cmap="gray"); plt.title("Mask"); plt.axis("off")
ov = img_disp.copy(); ov[mask>0,0] = np.clip(ov[mask>0,0] + 0.4, 0, 1)
plt.subplot(1,3,3); plt.imshow(ov); plt.title("Overlay"); plt.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# Vectorize mask to polygons in the GeoTIFF CRS
shapes_gen = features.shapes(mask, transform=transform)
geoms = [shape(g) for g, v in shapes_gen if v == 1]

gdf = gpd.GeoDataFrame({"geometry": geoms}, crs=crs).reset_index(drop=True)
gdf["building_id"] = np.arange(1, len(gdf)+1)

# Project to UTM for areas in m²
utm_crs = gdf.estimate_utm_crs()
gdf_utm = gdf.to_crs(utm_crs)
gdf["area_m2"] = gdf_utm.area

# Centroids in WGS84 (lat/lon)
to_wgs84 = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
centroids = gdf.geometry.centroid
lonlat = np.array([to_wgs84.transform(p.x, p.y) for p in centroids])
gdf["lon"] = lonlat[:,0]
gdf["lat"] = lonlat[:,1]

print("Buildings:", len(gdf))
gdf.head()


In [ ]:
# Choose building (largest roof by area)
b = gdf.sort_values("area_m2", ascending=False).iloc[0]
BID = int(b["building_id"])
print("Selected building_id:", BID, "area_m2:", b["area_m2"], "lat/lon:", (b["lat"], b["lon"]))

roof_poly = gdf.loc[gdf["building_id"]==BID, "geometry"].iloc[0]
neighbors = gdf[gdf["building_id"] != BID].copy()

# Show selected building on the image
# (plot in pixel space by rasterizing selection quickly)
sel_mask = features.rasterize([(roof_poly, 1)], out_shape=(H,W), transform=transform, fill=0, dtype=np.uint8)

plt.figure(figsize=(7,7))
ov2 = img_disp.copy()
ov2[sel_mask>0,1] = np.clip(ov2[sel_mask>0,1] + 0.5, 0, 1)  # highlight in green
plt.imshow(ov2); plt.title(f"Selected building #{BID}"); plt.axis("off")
plt.show()


In [ ]:
lat = float(b["lat"]); lon = float(b["lon"])

# One full year hourly (you can change the year)
times = pd.date_range("2025-01-01", "2025-12-31 23:00:00", freq="1h", tz=TZ)

location = pvlib.location.Location(latitude=lat, longitude=lon, tz=TZ, altitude=ALTITUDE_M)

# Clear-sky irradiance: ghi, dni, dhi
cs = location.get_clearsky(times, model="ineichen")  # :contentReference[oaicite:5]{index=5}

# Solar position
solpos = location.get_solarposition(times)

cs.head(), solpos.head()


In [ ]:
def poa_irradiance(surface_tilt, surface_azimuth, solpos, cs):
    # Transpose irradiance to plane-of-array
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=surface_tilt,
        surface_azimuth=surface_azimuth,
        solar_zenith=solpos["apparent_zenith"],
        solar_azimuth=solpos["azimuth"],
        dni=cs["dni"],
        ghi=cs["ghi"],
        dhi=cs["dhi"],
        model="haydavies"  # pvlib default commonly used in examples :contentReference[oaicite:7]{index=7}
    )
    return poa

def cell_temperature(poa_global, temp_air=20.0, wind_speed=1.0):
    # Use SAPM temperature model parameters (common default in pvlib examples) :contentReference[oaicite:8]{index=8}
    params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS["sapm"]["open_rack_glass_glass"]
    tcell = pvlib.temperature.sapm_cell(poa_global, temp_air, wind_speed, **params)
    return tcell

def pvwatts_power(poa_effective, tcell, pdc0_w, gamma_pdc=GAMMA_PDC, inv_eff=INV_EFF_NOM):
    # DC
    pdc = pvlib.pvsystem.pvwatts_dc(poa_effective, tcell, pdc0=pdc0_w, gamma_pdc=gamma_pdc)  # :contentReference[oaicite:9]{index=9}
    # AC (simple inverter limit + efficiency)
    pac0 = pdc0_w  # assume inverter sized to DC (simple)
    pac = pvlib.inverter.pvwatts(pdc, pdc0=pac0, eta_inv_nom=inv_eff)  # :contentReference[oaicite:10]{index=10}
    return pdc, pac


In [ ]:
def building_height_m(area_m2):
    if not HEIGHT_BY_AREA:
        return DEFAULT_BUILDING_HEIGHT_M
    # crude mapping by footprint area -> height
    # (tunable; just to introduce the factor)
    h = HEIGHT_MIN_M + (HEIGHT_MAX_M - HEIGHT_MIN_M) * (area_m2 / (area_m2 + 500))
    return float(np.clip(h, HEIGHT_MIN_M, HEIGHT_MAX_M))

def bearing_deg(p_from, p_to):
    # bearing from p_from to p_to in CRS meters (x east, y north)
    dx = p_to.x - p_from.x
    dy = p_to.y - p_from.y
    ang = np.degrees(np.arctan2(dx, dy))  # 0=N, 90=E
    return (ang + 360) % 360

def ang_diff(a, b):
    d = abs(a - b) % 360
    return min(d, 360 - d)

def shading_factor_for_point(point_xy, roof_h, neighbors_gdf, solpos, max_check=50):
    """
    Returns Series in [0,1] where 0 means fully shaded (direct blocked),
    1 means direct not blocked. Diffuse stays.
    """
    # Preselect nearest neighbors to speed up
    # Work in UTM for distance/geometry
    # neighbors_gdf must be in same projected CRS as point_xy
    neigh = neighbors_gdf.copy()
    neigh["dist"] = neigh.geometry.distance(point_xy)
    neigh = neigh.sort_values("dist").head(max_check)

    sun_az = solpos["azimuth"].values
    sun_el = solpos["apparent_elevation"].values

    shaded = np.zeros(len(solpos), dtype=bool)

    for _, row in neigh.iterrows():
        poly = row.geometry
        # nearest point on polygon boundary (approx obstacle direction)
        near_pt = poly.representative_point()
        b = bearing_deg(point_xy, near_pt)
        d = max(row["dist"], 1.0)

        h_obs = building_height_m(row.get("area_m2", 200))
        dh = max(h_obs - roof_h, 0.0)

        # obstacle vertical angle as seen from roof point
        obstacle_angle = np.degrees(np.arctan2(dh, d))  # deg

        # obstacle "angular width" approximation
        # radius ~ sqrt(area/pi), width angle ~ arctan(r/d)
        r = np.sqrt(max(row.get("area_m2", 100), 10) / np.pi)
        half_width = np.degrees(np.arctan2(r, d))

        # block if: sun comes from that direction AND sun elevation below obstacle angle
        dir_match = (np.array([ang_diff(sa, b) for sa in sun_az]) <= half_width)
        elev_block = (sun_el <= obstacle_angle)

        shaded |= (dir_match & elev_block & (sun_el > 0))

    # 1 = no block, 0 = blocked (direct)
    return pd.Series((~shaded).astype(np.float32), index=solpos.index)


In [ ]:
# Prepare geometries in UTM for shading distances
roof_gdf = gpd.GeoDataFrame({"geometry":[roof_poly], "area_m2":[float(b["area_m2"])]}, crs=crs).to_crs(utm_crs)
neighbors_utm = neighbors.to_crs(utm_crs)
neighbors_utm["area_m2"] = neighbors.to_crs(utm_crs).area  # ensure

roof_centroid = roof_gdf.geometry.iloc[0].centroid
roof_area_m2 = float(roof_gdf["area_m2"].iloc[0])

# installed DC size estimate from usable area
usable_area = roof_area_m2 * ROOF_USABLE_FRACTION
pdc0_w = usable_area * POWER_DENSITY_W_PER_M2

roof_h = building_height_m(roof_area_m2)  # assumed roof height

# shading factor time series (1=no shade on direct)
if USE_SHADING:
    sf = shading_factor_for_point(roof_centroid, roof_h, neighbors_utm, solpos, max_check=60)
else:
    sf = pd.Series(1.0, index=solpos.index)

def annual_energy_kwh(tilt, azimuth):
    poa = poa_irradiance(tilt, azimuth, solpos, cs)
    # apply shading only to direct component (simple)
    poa_eff = poa["poa_global"].copy()
    # better: reduce poa_direct, keep diffuse; we approximate on global:
    poa_eff = poa["poa_diffuse"] + poa["poa_direct"] * sf

    tcell = cell_temperature(poa_eff, temp_air=20.0, wind_speed=1.0)
    pdc, pac = pvwatts_power(poa_eff, tcell, pdc0_w, gamma_pdc=GAMMA_PDC, inv_eff=INV_EFF_NOM)

    # apply generic system losses
    pac_net = pac * (1.0 - SYSTEM_LOSS_FRACTION)

    # Wh -> kWh (hourly samples -> sum of W over 1h = Wh)
    annual_kwh = pac_net.sum() / 1000.0
    return float(annual_kwh)

# Sweep
results = []
for tilt in TILT_GRID:
    for az in AZI_GRID:
        e = annual_energy_kwh(tilt, az)
        results.append((tilt, az, e))

df = pd.DataFrame(results, columns=["tilt_deg","azimuth_deg","annual_kwh"])
best = df.sort_values("annual_kwh", ascending=False).iloc[0]
best


In [ ]:
best_tilt = float(best["tilt_deg"])
best_az   = float(best["azimuth_deg"])
best_kwh  = float(best["annual_kwh"])

kwp = pdc0_w / 1000.0
specific_yield = best_kwh / kwp if kwp > 0 else np.nan

print("====== PV Study (Clear-sky, with simple shading) ======")
print(f"Building ID: {BID}")
print(f"Roof area: {roof_area_m2:.1f} m² | usable: {usable_area:.1f} m²")
print(f"Estimated DC size: {kwp:.2f} kWp (power_density={POWER_DENSITY_W_PER_M2} W/m²)")
print(f"Best tilt: {best_tilt:.0f}° | best azimuth: {best_az:.0f}° (0=N, 90=E, 180=S)")
print(f"Annual energy: {best_kwh:.0f} kWh/year")
print(f"Specific yield: {specific_yield:.0f} kWh/kWp/year")
print(f"Assumptions: losses={SYSTEM_LOSS_FRACTION*100:.0f}%, eff={MODULE_EFF*100:.0f}%, shading={'ON' if USE_SHADING else 'OFF'}")

# Heatmap quick (tilt vs az)
pivot = df.pivot_table(index="tilt_deg", columns="azimuth_deg", values="annual_kwh")
plt.figure(figsize=(12,4))
plt.imshow(pivot.values, aspect="auto")
plt.xticks(range(len(pivot.columns)), pivot.columns, rotation=90)
plt.yticks(range(len(pivot.index)), pivot.index)
plt.colorbar(label="annual kWh")
plt.title("Annual energy (kWh) sweep: tilt vs azimuth")
plt.xlabel("Azimuth (deg)")
plt.ylabel("Tilt (deg)")
plt.tight_layout()
plt.show()


In [ ]:
from shapely.prepared import prep

def sample_points_in_polygon(poly, n=200):
    # uniform sampling in bbox then keep points inside
    minx, miny, maxx, maxy = poly.bounds
    P = prep(poly)
    pts = []
    tries = 0
    while len(pts) < n and tries < n*50:
        tries += 1
        x = np.random.uniform(minx, maxx)
        y = np.random.uniform(miny, maxy)
        p = Point(x,y)
        if P.contains(p):
            pts.append(p)
    return pts

poly_utm = roof_gdf.geometry.iloc[0]
pts = sample_points_in_polygon(poly_utm, n=250)

# Score each point by how often direct is unshaded (sum(sf))
scores = []
for p in pts:
    sf_p = shading_factor_for_point(p, roof_h, neighbors_utm, solpos, max_check=60) if USE_SHADING else pd.Series(1.0, index=solpos.index)
    scores.append(sf_p.mean())  # fraction of time unshaded (direct)
scores = np.array(scores)

best_idx = int(np.argmax(scores))
best_point = pts[best_idx]
print("Best point shading score:", scores[best_idx])

# Plot roof + points (UTM)
fig, ax = plt.subplots(1,1, figsize=(6,6))
gpd.GeoSeries([poly_utm]).plot(ax=ax, facecolor="none", edgecolor="black")
ax.scatter([p.x for p in pts], [p.y for p in pts], s=8, c=scores, cmap="viridis")
ax.scatter([best_point.x], [best_point.y], s=80, c="red", marker="x")
ax.set_title("Best PV placement (proxy) - higher score = less shading")
ax.set_axis_off()
plt.show()


In [ ]:
out_geojson = os.path.join(OUTPUT_DIR, f"pv_building_{BID}.geojson")
out_csv = os.path.join(OUTPUT_DIR, f"pv_building_{BID}.csv")

# attach results to the building feature
gdf_one = gdf[gdf["building_id"]==BID].copy()
gdf_one["roof_area_m2"] = roof_area_m2
gdf_one["usable_area_m2"] = usable_area
gdf_one["pdc0_kwp"] = kwp
gdf_one["best_tilt_deg"] = best_tilt
gdf_one["best_azimuth_deg"] = best_az
gdf_one["annual_kwh"] = best_kwh
gdf_one["specific_yield_kwh_per_kwp"] = specific_yield
gdf_one.to_file(out_geojson, driver="GeoJSON")

pd.DataFrame([{
    "building_id": BID,
    "lat": lat, "lon": lon,
    "roof_area_m2": roof_area_m2,
    "usable_area_m2": usable_area,
    "pdc0_kwp": kwp,
    "best_tilt_deg": best_tilt,
    "best_azimuth_deg": best_az,
    "annual_kwh": best_kwh,
    "specific_yield_kwh_per_kwp": specific_yield,
    "loss_fraction": SYSTEM_LOSS_FRACTION,
    "shading_enabled": USE_SHADING
}]).to_csv(out_csv, index=False)

print("✅ Saved:", out_geojson)
print("✅ Saved:", out_csv)
